In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Install dependencies (this is mostly for Google Colab, as the other dependences are available by default in Colab)
try:
  import datasets, evaluate, accelerate
  import gradio as gr
except ModuleNotFoundError:
  !pip install -U datasets evaluate accelerate gradio # -U stands for "upgrade" so we'll get the latest version by default
  import datasets, evaluate, accelerate
  import gradio as gr

import random

import numpy as np
import pandas as pd

import torch
import transformers

print(f"Using transformers version: {transformers.__version__}")
print(f"Using datasets version: {datasets.__version__}")
print(f"Using torch version: {torch.__version__}")

Using transformers version: 4.49.0
Using datasets version: 3.4.1
Using torch version: 2.6.0+cu126


In [5]:
dataset = datasets.load_dataset(path="mrdbourke/learn_hf_food_not_food_image_captions")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
})

In [12]:
len(dataset["train"])

250

In [13]:
range(len(dataset["train"]))

range(0, 250)

In [ ]:
#visualizing the data 

import random
random_indexs = random.sample(range(len(dataset["train"])),5)
random_samples = dataset["train"][random_indexs]

print("INFO: Random sample from the dataset\n")
for item in zip(random_samples["text"],random_samples["label"]):
    print(f"Text: {item[0]} | Label: {item[1]}")

INFO: Random sample from the dataset

Text: Red ceramic mug sitting next to a coffee maker | Label: not_food
Text: Set of measuring spoons hung on a rack | Label: not_food
Text: Wooden dresser with a mirror reflecting the room | Label: not_food
Text: Nigiri sushi topped with fresh salmon and tuna slices on vinegared rice. | Label: food
Text: Vegetarian sushi roll with avocado and pickled radish filling. | Label: food
